Videocvičení naleznete zde: https://youtu.be/FLCeIwfrmxI

In [ ]:
!pip install ipympl

In [ ]:
%cd /content
%rm -R SKOMAM
!git clone https://github.com/Beremi/SKOMAM
%cd /content/SKOMAM/past_SKOMAM/2021/CV3/
%pwd

# Simulace šíření nákazy - zjednodušený 2D model

### Načítání balíčků
Budeme používat knihovnu **numpy** s aliasem **np** pro matematické funkce a práci s poli a knihovnu **matplotlib** s aliasem **plt** pro vykreslování výsledků. Pro pěknou vizualizaci průběhu simulace si také vyzkoušíme animace **FuncAnimation** z knihovny matplotlib. 

Následujícím kódem naimportujeme balíčky. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

Pro interaktivní vykreslování grafů v Jupyter notebooku ještě potřebujeme toto:

In [ ]:
%matplotlib widget
from google.colab import output
output.enable_custom_widget_manager()

### Principy simulace
Budeme simulovat časový vývoj šíření nemoci v populaci. Nejprve si nastavíme dva parametry: **pocet_lidi** a **delka_simulace** v minutách. Parametry **pocet_lidi** a **delka_simulace** můžeme později pro účely testování libovolně měnit. Jako další parameter (avšak fixní, výrazně to zjednoduší naši práci) budeme uvažovat časový krok simulace, jako jeho hodnotu budeme považovat 15 minut.

In [ ]:
pocet_lidi = 1000 # nezadavejte hodnoty vetsi nez 5000, simulace pak bude velmi pomala
delka_simulace = 30*24*60 # 30 dni * 24 hodin * 60 minut

Princip naší simulace bude následující:
- jednotliví lidé budou reprezentováni body ve 2D
- tyto body se budou v čase (jednotlivých časových krocích) pohybovat - měnit své pozice
- pokud je někdo nakažen, může nakazit lidi ve své blízkosti

### SIR model
SIR model popisuje základní principy šíření nemoci. Je založen na rozdělení populace do ří skupin:
1. **S** (susceptible) - lidé, kteří doposud nepřišli do styku s nemocí a mohou se nakazit
2. **I** (infected) - nakažení jedinci, mohou nakazit lidi ze skupiny **S**
3. **R** (removed) - jedinci, kteří se už nemohou nakazit (ať už v dobrém - imunita, či zlém - smrt). Budeme uvažovat, že po pevně dané době se z **I** stává **R**


<hr style="border:1px solid black"> </hr>

## Úkol 1: 
Vygenerujte pro každého jedince v populaci o velikosti **pocet_lidi** náhodnou pozici v oblasti $\langle 0,1 \rangle \times \langle 0,1 \rangle$ a vykreslete celou populaci.

<hr style="border:1px solid black"> </hr>

In [ ]:
pozice = np.random.rand(pocet_lidi,2)
plt.figure()
plt.plot(pozice[:,0],pozice[:,1],"bo",markersize=2)

Dále vyrobíme numpy pole, které bude identifikovat jednotlivé skupiny **S, I, R**. Ze začátku budeme považovat všechny ve stavu **S**.

In [ ]:
stav = np.full(pocet_lidi, 'S')

V dalších částech budeme chtít vybírat z polí parametrů (například pozice) pouze hodnoty příslušné ke konkrétní skupině (S, I, R). To se dá jednoduše takto:

In [ ]:
pozice_S = pozice[stav == 'S', :]

<hr style="border:1px solid black"> </hr>

## Úkol 2: 
Uvažujme, že každý jedinec v populaci má jednoznačný index. Převeďte osoby s indexem 0-99 z S do I a osoby s indexem 100-299 z S do R a vykreslete celou populaci (pro každý stav volte jinou barvu). Například: S-modrá="bo", I-červená="ro", R-zelená="go".

<hr style="border:1px solid black"> </hr>

In [ ]:
# změňte stavy (nezapomeňte, že "a:b" vybírá celá čísla a<=n<b)
stav[0:100] = 'I'
stav[100:300] = 'R'
# vykreslete
plt.figure()
# tři grafy (plot), každý pro jednu barvu
plt.plot(pozice[stav == 'S', 0], pozice[stav == 'S', 1], "bo", markersize=2)
plt.plot(pozice[stav == 'I', 0], pozice[stav == 'I', 1], "ro", markersize=2)
plt.plot(pozice[stav == 'R', 0], pozice[stav == 'R', 1], "go", markersize=2)

### Vykreslování průběžných výsledků - Animace
Abychom se mohli dostat dále, ukážeme si, jak se dělají animace. Budeme používat funkci **FuncAnimation**, která pracuje následovně:
- má 3 základní parametry:
 - objekt figure z matplotlib (**fig**)
 - námi vytvořená funkce, říkejme jí **update**
 - iterovatelé pole (**frames**), případně počet framů (pokud je vstupem iterovatelné pole, pak je počet framů velikost tohoto pole)
- pro nás nejdůležitější bude funkce **update**, projděme si tedy její vlastnosti:
 - jako vstup bude dostávat jednotlivé položky vstupu **frames**
 - zavolá se jednou pro každou položku **frames**, bude tedy obsahovat celou naši simulaci jednoho časového kroku
 - bude obsahovat aktualizace součástí grafů, které budeme chtít mezi jednotlivými framy měnit
 
Abyste si udělali lepší představu, ukážeme demostraci:

Nejprve vyrobení grafu, do kterého budeme vykreslovat animaci. Pro jednoduché znovupoužití bez kopírování si na to napíšeme funkci. Jelikož budeme chtít vykreslovat mnoho věcí tak je tako funkce poměrně dlouhá. Proto ji máme v jiném souboru s pomocnými funkcemi. Zájemci se na ní mohou podívat (pomocne_funkce.py).

Okno s grafy bude vypadat následovně:
- **levé okno bude obsahovat aktuální dění v simulaci - pohyb teček a jejich barevné určení**
- **do pravého okna budeme vykreslovat časový vývoj simulace v podobě počtu lidí v jednotlivých skupinách**
 - modrá = S
 - červená = I
 - zelená = R
 - pro pozdější využití zavedeme také karanténu = černá = Q
 
 Funkci naimportujeme tímto kódem:

In [ ]:
from pomocne_funkce import nova_animace

Funkce **nova_animace** vrací dvě hodnoty: graf a tuple všechn komponentů grafu, které později budeme chtít upravit/aktualizovat v průběhu animace. Jako parametry vkládáme numpy pole pozic a stavů, velikost populace a délku simulace.

In [ ]:
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)

Dále vyrobíme zmíněnou funkci **update**, pro jednodušší použití v dalších úkolech si vyrobíme zvlášť funkci, která bude obstarávat vykreslování (jak barevných teček, tak času simulace), nazveme ji **vykresleni**.

Funkce **vykresleni** bude upravovat data komponent grafu (jednotlivé čáry, případně text) dle současného stavu simulace. Opět ji máme pro větší přehlednost tohoto notebooku implementovanou v souboru pomocných funkcí (zájemci se mohou podívat).

Funkci naimportujeme následovně:

In [ ]:
from pomocne_funkce import vykresleni

Funkce vykreslení bude pouze upravovat objekty (komponenty grafu) které už existují, takže nebude nic vracet. Jako své vstupy bude brát aktuální čas simulace, pozice, stav, komponenty grafu:

Konečně přistoupíme k funkci **update**, ta bude díky předchozí funkci velmi jednoduchá a přímočará. U funkce update budeme pro jednoduchost adresovat přímo promměné v globální paměti (mimo scope samotné funkce). Toto je možné vždy, ačkoliv to není doporučováno kvůli čitelnosti kódu a případným chybám. My k tomuto přistoupíme, protože nám to zjednoduší implementaci samotné animace.

In [ ]:
def update(cas_simulace_minuty): #na vstupu budeme uvažovat aktuální čas
    # všechny proměnné definované vně této funkce jsou pro nás stále přístupné (pozice, stav, ...)
    # upravíme pozice, ať vidíme že se něco děje 
    # pozor na: pozice =... v tu chvíli bychom definovali novou lokální proměnnou
    pozice[:,:] += (np.random.rand(pocet_lidi,2)-0.5)/100 # přičteme k nim náhodné číslo mezi -0.005 s 0.005
    # zavolání vykreslovací funkce (až potom co upravíme pozice a stav)
    vykresleni(cas_simulace_minuty, pozice, stav, komponenty_grafu) 

Zbývá už jen spustit animaci funkcí **FuncAnimation**, první parmetr je figure, ve kterém se bude vše dít, pak updatovací funkce zajišťující samotnou animaci, dále proměnná frames obsahující sekvenci všech časů (zde použijeme fixní časový krok 15 minut), ve kterých bude funkce update spuštěna. Dodatečných parametrů si nemusíte všímat, pokud by vás však zajímalo, co dělají pak:
- interval - pauza mezi framy v milisekundách a 
- repeat - zda chceme opakovat pořád dokola.

**Pozor, animace poběží v okně nahoře, tam kde se prvně vykreslila při inicializaci. Simulaci můžete kdykoliv ukončit vypínacím tlačítkem vpravo nahoře.**

In [ ]:
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1, repeat = False)
plt.show()

Pokud animace běží moc rychle můžetevyzkoušet změnit **interval** na větší hodnotu.


#### Nezapomeňte, že veškeré proměnné z dřívějších inputů jsou stále aktivní - restart/reinicializace vstupních dat
Jelikož si nyní začneme hrát se simulací, bude se nám hodit funkce, která vyresetuje všechny parametry simulace. Jako základní stav budeme uvažovat všechny lidi ve stavu **S** kromě prvních 5 (index 0 - 4), kteří budou ve stavu **I**.

Abychom tuto funkci nemuseli později přepisovat, rovnou přidáme další pole, která později využijeme při rozšiřování naší simulace.

In [ ]:
def reset_simulace(pocet_lidi):
    pozice = np.random.rand(pocet_lidi,2) # nové náhodné pozice
    doba_od_nakazeni = np.zeros((pocet_lidi)) # pole kde budeme udržovat dobu od prvotního nakažení
    nakazil_lidi = np.zeros((pocet_lidi)) # pole, kde budeme napočítávat kolik lidí kdo nakazil
    stav = np.full(pocet_lidi,'S') # inicializace stavu
    stav[0:4] = 'I' # první 4 (indexy 0,1,2,3) budou nakažení
    return pozice, stav, doba_od_nakazeni, nakazil_lidi

Budeme jí pak volat takto:

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi = reset_simulace(pocet_lidi)

Můžeme si zkontrolovat výsledek pomocí spuštění nové animace:

In [ ]:
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1,repeat=False)
plt.show()

### Implementace nákazy
Abychom mohli pokročit dál, budeme muset naprogramovat průběh nákazy. Jedním z důležitých faktorů je vzdálenost mezi nakaženým **I** a nakazitelným **S** jedincem.

Abychom si toto mohli lépe ozkoušet, nastavíme nákazu dalším 5 lidem a ukážeme si, jak jednoduše dostat pole indexů s konkrétními stavy.

In [ ]:
stav[[7, 12, 15, 6, 3]] = 'I'
idx_I, = (stav == 'I').nonzero() # funkce vrací dvě hodnoty a nás zajímá pouze první, proto je tam "," a za ní nic
print(idx_I)

Jsou tam také indexy 0-3, protože jsme je měli už v inicializaci.

Řekněme, že lidé se mohou nakazit, pokud jsou ve vzdálenosti menší než 0.02 od infikovaného.

In [ ]:
vzdalenost_nakazy = 0.02
mohl_se_nakazit = np.full(pocet_lidi, False)

<hr style="border:1px solid black"> </hr>

## Úkol 3: 
Vypočtěte vzdálenosti mezi nakaženými (osoby ve stavu **I**) a nakazitelnými (osoby ve stavu **S**). Pokud má nakazitelný vzdálenost od některého nakaženého menší než hodnota proměnné **vzdalenost\_nakazy**, pak vepište do příslušné pozice v poli **mohl\_se\_nakazit** hodnotu True.

<hr style="border:1px solid black"> </hr>

In [ ]:
# indexy lidi ve stavu S a stavu I
idx_I, = (stav == 'I').nonzero() # funkce vrací 2 hodnoty, my chceme pouze první z nich
idx_S, = (stav == 'S').nonzero()
# dvě vnořené smyčky skrze indexy, ať projdeme všechny dvojice
# pro každou dvojici pak spočteme vzdálenost a pokud je menší než vzdalenost_nakazy, upravíme mohl_se_nakazit
for i in idx_I:
    for j in idx_S:
        rozdil = pozice[i, :] - pozice[j, :]
        distance = np.sqrt(rozdil[0]**2 + rozdil[1]**2)
        if distance <= vzdalenost_nakazy:
            mohl_se_nakazit[j] = True

Pro kontrolu si vykresleme nakažené jedince a ty z **S**, kteří se nakazit mohli:

In [ ]:
plt.figure()
# vykreslíme nakažené (stav == 'I')
plt.plot(pozice[stav == 'I', 0], pozice[stav == 'I', 1], "ro")
# vykreslíme ty co by se mohli nakazit (stav == 'S') a jsou zároveň dost blízko (mohl_se_nakazit)
plt.plot(pozice[(stav == 'S') & mohl_se_nakazit, 0], pozice[(stav == 'S') & mohl_se_nakazit, 1], "bo")

Doufám, že se vše zdařilo, pokud ne, podívejte se do řešení, bez této části se totiž dále nedostaneme.

#### Pravděpodobnost nakažení při kontaktu s nakaženým
Jelikož ne vždy, když se dostanete do blízkosti nakaženého, se hned nakazíte. Proto budeme chtít toto zanést i do našeho modelu. Jelikož budeme pracovat s 15 minutovými časovými kroky, můžeme pracovat s pravděpodobností nákazy za 15 minut v blízkosti někoho nakaženého, jako tuto pravděpodobnost můžeme vzít například 5 %. 

To znamená, že pokud se po dobu 15 minut (náš časový krok) pohybujete v blízkosti nakaženého, je 5 % šance, že se nakazíte. **Tato šance je pro každý kontakt s nakaženým a při více kontaktech se aplikuje vícekrát**, asi souhasite s tím, že při pohybu mezi 10 nakaženými je šance nákazy výrazně větší, než při pohybu v blízkost jednoho nakaženého. (nikoliv však 10x větší - můžete si rozmyslet) .

In [ ]:
pravdepodobnost_nakazy = 0.05
np.random.rand() < pravdepodobnost_nakazy # takto vyrobíme náhodný pokus, kde s pravděpodobností 5 % výjde True

<hr style="border:1px solid black"> </hr>

## Úkol 4: 
implementujte funkci **nove_nakazeni**, ve které je pro každého ze skupiny **S** v kontaktu s každým nakaženým **I** aplikována šance na nákazu **pravdepodobnost_nakazy**. Výstupem budou dvě pole: pole nově nakažených **nakazili_se** a pole přírůstku lidí, které infikovaní nakazili **nakazili_ostatni**.
- Pro každou dvojici S-I spočítáte vzdálenost a zjistíte, zda je vzdálenost kratší než **vzdalenost_nakazy** (použijte kód z předchozího úkolu)
- Pokud ano, aplikujete **pravdepodobnost_nakazy** -> True hodnota znamená, že se jedinec S nakazil.
 - tedy True hodnota do pole **nakazili_se** pro daného jedince z **S** který byl nakažen a +1 do pole **nakazili_ostatni** pro jedince z **I** který byl tím nakazitelem.

<hr style="border:1px solid black"> </hr>

In [ ]:
def nove_nakazeni(pozice, stav, vzdalenost_nakazy, pravdepodobnost_nakazy):
    # nainicializujte si pole nakazili_se na hodnoty False (do něj pak přidáme True pokud se někdo nakazí)
    nakazili_se = np.full(pocet_lidi,False)
    # nainicializujte si pole nakazili_ostatni na 0 (do něj pak přidáme +1 za každého koho daný jedinec nakazil)
    nakazili_ostatni = np.full(pocet_lidi,0)    
    # indexy jednotlivých I a S
    idx_I, = (stav == 'I').nonzero()
    idx_S, = (stav == 'S').nonzero()
    # smyčka přes všechny dvojice
    for i in idx_I:
        for j in idx_S:
            rozdil = pozice[i,:]-pozice[j,:]
            distance = np.sqrt(rozdil[0]**2 + rozdil[1]**2)
            if distance <= vzdalenost_nakazy:
                if np.random.rand() < pravdepodobnost_nakazy:
                    nakazili_se[j] = True
                    nakazili_ostatni[i] +=1
    return nakazili_se, nakazili_ostatni # vrátíme pozadovaná pole

Nyní můžeme upravit funkci **update**, tak aby používala nově vyrobenou funkci:

In [ ]:
def update(cas_simulace_minuty):
    pozice[:,:] += (np.random.rand(pocet_lidi,2)-0.5)/100 #totéž co v předchozí funkci
    
    # zavoláme naši novou funkci
    nakazili_se, nakazili_ostatni = nove_nakazeni(pozice, stav, vzdalenost_nakazy, pravdepodobnost_nakazy)
    
    stav[nakazili_se] = 'I' # změna stavu dle nových nákaz
    nakazil_lidi[:] += nakazili_ostatni # přičtení kolik kdo nakazil nových lidí v tomto časovém kroce
    
    vykresleni(cas_simulace_minuty, pozice, stav, komponenty_grafu) 
    return

Samotná animace bohužel neukáže, pokud má funkce **update** nebo její součást nějakou chybu, proto je lepší vyzoušet ji samostatně (pokud se nic nestane, znamená to, že jsme neudělali chybu v syntaxi a funce proběhla bez pádu).

In [ ]:
update(0) # nebo update(1000) pro pozdější časový krok

A spustíme simulaci, ať vidíme, co se nám povedlo:

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi = reset_simulace(pocet_lidi)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1, repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()

Gratulujeme, pokud vám to běží. Nicméně jste si pravděpodobně všimli, že je to nyní velice pomalé. Je to tím, že i takto jednoduchá simulace vyžaduje mnoho výpočtů, které navíc nyní děláme velice neefektivně. **Simulaci můžete vypnout tlačítkem vpravo nahoře animovaného okna (nebo kódem v předchozí buňce).**

#### Efektivní implementace
Výpočty se dají dělat mnohem efektivněji pokud použijeme maticové operace poskytované balíčkem numpy. V souboru pomocne_funkce.py jsme připravili efektivní variantu funkce **nove_nakazeni**. Zájemci si ji mohou prohlédnout.

Tímto ji načteme:

In [ ]:
from pomocne_funkce import nove_nakazeni

Načtením jsme hned přepsali původní funkci, takže zopakováním následujícího si hned funkci vyzkoušíme.

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi = reset_simulace(pocet_lidi)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1,repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()

Vidíme, že se naše nákaza velmi rychle šíří. Chybí nám však mechanizmus, který převede infekční jedince na jedince bez nákazy a zároveň jedince dále nenakazitelné.

### Zbavení se nákazy / přechod do stavu R

To vyřešíme pomocí počítadla doby nákazy:

In [ ]:
delka_nakazy = 7*24*15
doba_od_nakazeni = np.zeros((pocet_lidi)) # mimojiné toto pole již vytváříme v naší funkci restart_simulace

<hr style="border:1px solid black"> </hr>

## Úkol 5: 
Přidejte do funkce **update** načítání času pro nakažené. Dále v každém kroku kontrolujte, jestli čas nepřekročil hodnotu **delka\_nakazy**, která udává, po jaké době se osoba ve stavu **I** přesune do stavu **R**. Pokud čas uvedenou hodnotu překročí, přesuňte osobu z **I** do **R**.

<hr style="border:1px solid black"> </hr>

In [ ]:
def update(cas_simulace_minuty):
    #totéž co v předchozí funkci
    pozice[:, :] += (np.random.rand(pocet_lidi, 2) - 0.5) / 100   # hrubá simulace náhodného pohybu
    nakazili_se, nakazili_ostatni = nove_nakazeni(pozice, stav, vzdalenost_nakazy, pravdepodobnost_nakazy)
    nakazil_lidi[:] += nakazili_ostatni # přičtení kolik kdo nakazil nových lidí v tomto časovém kroce
    stav[nakazili_se] = 'I' # změna stavu nově nakažených
    
    #----------------------------------------------------------------------------------------
    # inkrementujte dobu_od_nakazeni pro lidi se stavem 'I'
    doba_od_nakazeni[stav == 'I'] +=15
    # ti, kteří jsou nemocní a překonají delka_nakazy se změní na stav 'R'
    po_nemoci = (doba_od_nakazeni >= delka_nakazy) & (stav == 'I') # nemocní a překonají delka_nakazy
    stav[po_nemoci] = 'R'
    #----------------------------------------------------------------------------------------
    
    vykresleni(cas_simulace_minuty, pozice, stav, komponenty_grafu) 
    return

In [ ]:
update(0) # nebo update(1000) pro pozdější časový krok

Vyzkoušíme nově vytvořenou funkci: (vyzkoušejte si to spustit vícekrát, vzhledem k náhodnému generování výsledků to bude mít pokaždé jiný průběh),

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi = reset_simulace(pocet_lidi)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1, repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()

<hr style="border:1px solid black"> </hr>

## Úkol 6: 
Přidejte do funkce **update** výpočet reprodukčního čísla $R_0$ (v kódu nazvěte **R0**). Reprodukčního číslo $R_0$ spočítejte jako průměr počtu lidí, které každý člověk s ukončenou nákazou v tomto časovém kroku (**po_nemoci**) nakazil. K tomuto použíjte pole **nakazil_lidi**. Pokud není žádný takovýto člověk, nastavte $R_0=-1$ (vykreslovací kód toto vyhodnotí jako příkaz k neaktualizaci hodnoty).

<hr style="border:1px solid black"> </hr>

In [ ]:
def update(cas_simulace_minuty):
    #totéž co v předchozí funkci
    pozice[:, :] += (np.random.rand(pocet_lidi, 2) - 0.5) / 100   # hrubá simulace náhodného pohybu
    nakazili_se, nakazili_ostatni = nove_nakazeni(pozice, stav, vzdalenost_nakazy, pravdepodobnost_nakazy)
    nakazil_lidi[:] += nakazili_ostatni # přičtení kolik kdo nakazil nových lidí v tomto časovém kroce
    stav[nakazili_se] = 'I' # změna stavu nově nakažených
    
    doba_od_nakazeni[stav == 'I'] +=15 # inkrementujte dobu_od_nakazeni pro lidi se stavem 'I'
    po_nemoci = (doba_od_nakazeni >= delka_nakazy) & (stav == 'I') # nemocní a překonají delka_nakazy
    stav[po_nemoci] = 'R' # změní svůj stav na R
    
    #----------------------------------------------------------------------------------------
    vyber_poctu_nakazenych = nakazil_lidi[po_nemoci]
    if vyber_poctu_nakazenych.size == 0:
        R0 = -1
    else:
        R0 = vyber_poctu_nakazenych.mean()
    #----------------------------------------------------------------------------------------
    
    vykresleni(cas_simulace_minuty, pozice, stav, komponenty_grafu, R0) # R0 přidáme jako další parametr  
    return

In [ ]:
update(0) # nebo update(1000) pro pozdější časový krok

Vyzkoušíme nově vytvořenou funkci:

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi = reset_simulace(pocet_lidi)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1, repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()

### Vyzkoušejte si různé parametry simulace
Pokud vše běží, můžete si zkusit 'pohrát' s parametry simulace. Zkuste například zvýšit pravěpobnost nákazy.

**Pokud bychom chtěli nějak napodobovat opatření, která teď zažíváme, tak můžeme uvažovat například následující:**
- **Nošení roušek/zvýšená hygiena sníží pravděpodobnost nákazy.**
- **Snaha o vyhýbání se ostatním lidem / limitování osobních kontaktů může malinko zmenšit vzdálenost, na kterou se tečky mohou nakazit.**


In [ ]:
pocet_lidi=1000 #nezadavejte hodnoty vetsi nez 5000, simulace pak bude velmi pomala
delka_simulace=30*24*60 #30 dni * 24 hodin * 60 minut
vzdalenost_nakazy = 0.02 #vzdalenost, na kterou se mužeme nakazit
pravdepodobnost_nakazy = 0.01 #pravděpodobnost, že se nakazíme
delka_nakazy = 7*24*15 #délka nákazy

pozice, stav, doba_od_nakazeni, nakazil_lidi = reset_simulace(pocet_lidi)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1, repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()

### Více realistické tečky - pohyb mezi místy

Asi budete souhlasit s tím, že náhodně kmitající tečky nejsou dobrou aproximací pohybu lidí. Nyní zkusíme naši simulaci v tomto ohledu vylepšit. Budeme předpokládat, že každá tečka má nějaký domov a nějaké pracoviště. Podstatou tohoto je vynutit více náhodných kontaktů mezi tečkami se stejným domovem, případně stejným pracovištěm.

Nejprve si vyrobíme lokace domovů a pracovišť pro naše tečky (k tomu použijeme další z předpřipravených funkcí v souboru pomocne_funkce.py):

In [ ]:
grid_domovu = (4, 10) # velikost gridu domacnosti
grid_praci = (2, 5) # velikost gridu praci

from pomocne_funkce import pozice_domovu_praci

prace, domovy = pozice_domovu_praci(grid_domovu, grid_praci)

Pro lepší představu si pozice domovů a pracovišť vykreslíme:

In [ ]:
plt.figure()
plt.plot(domovy[:,0],domovy[:,1],"bo",markersize=2)
plt.plot(prace[:,0],prace[:,1],"ro",markersize=2)

Dále budeme chtít vyrobit dvě pole obdobné poli aktuálních pozic, která budou obsahovat pozice domovů a práce každého člověka/tečky.

Pro jednoduché znovuvyužití toto přidáme do funkce **reset_simulace**.
Použijeme funkci **random.randint** pro náhodné přiřazení pracoviště a domova pro každého:

In [ ]:
def reset_simulace(pocet_lidi, domovy, prace):
    pozice = np.random.rand(pocet_lidi,2) # nové náhodné pozice
    doba_od_nakazeni = np.zeros((pocet_lidi)) # pole kde budeme udržovat dobu od prvotního nakažení
    nakazil_lidi = np.zeros((pocet_lidi)) # pole, kde budeme napočítávat kolik lidí kdo nakazil
    stav = np.full(pocet_lidi,'S') # inicializace stavu
    stav[0:4] = 'I'
    
    # np.random.randint má tři parametry - nejnižší číslo, nejvyšší číslo, počet hodnot
    domovy_lidi = domovy[np.random.randint(0, domovy.shape[0], pocet_lidi),:]
    prace_lidi = prace[np.random.randint(0, prace.shape[0], pocet_lidi), :]
    
    return pozice, stav, doba_od_nakazeni, nakazil_lidi, domovy_lidi, prace_lidi

Nově budeme reset simulace volat takto (přibyly nám dvě nové výstupní hodnoty):

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi, domovy_lidi, prace_lidi = reset_simulace(pocet_lidi, domovy, prace)

<hr style="border:1px solid black"> </hr>

## Úkol 7: 
Naimplementujte funkci **do\_prace\_a\_zpet**, která bude splňovat následující:

- v čase od 6:00 do 14:00 se jedinci (znázornění tečkami) přesouvají/jsou v práci,
- ve zbytku času se jedinci přesouvají/jsou doma,
- rychlost přesunu jedinců je 0.1 v jednom časovém kroku,
- funkce vrací vektor posunu (změny pozice) jedinců,
- pokud jsou jedinci do vzdálenosti 0.2 od pracoviště, už se nemusí dále přesouvat,
- pokud jsou jedinci do vzdálenosti 0.05 od domova, už se nemusí dále přesouvat.

<hr style="border:1px solid black"> </hr>

In [ ]:
def do_prace_a_zpet(cas_simulace_minuty, pozice, domovy_lidi, prace_lidi):
    posun = np.zeros(pozice.shape) # inicializace výstupu
    # nezapomeňte, že cas_simulace_minuty je celkový čas. Jak z něj dostaneme čas od půlnoci dnes?
    cas_pres_den = np.mod(cas_simulace_minuty,24*60)
    # nastavte si promennou cil podle toho, jaká část dne je
    if (cas_pres_den >= 6*60) & (cas_pres_den <= 14*60):
        cil = prace_lidi
    else:
        cil = domovy_lidi
    # najděte směr k cíli
    smer = cil - pozice
    # napočítejte si vzdálenost k cíli
    vzdalenost = np.sqrt(smer[:,0]**2 + smer[:,1]**2)
    # nezapomeňtě, že směr by měl být jednotkový vektor (znormalizujte ho)
    # pokud chcete vyrobit nové pole (a né pouze odkaz ne jiné) musíte použít funkci .copy: a = b.copy()
    smer[:,0] /= vzdalenost
    smer[:,1] /= vzdalenost
    # napočítejte velikost, o kterou se ve směru k cíli posunete (maximálně 0.1 což je maximální rychlost)
    # může být 0, pokud jsme dostatečne blízko (v závislosti na cíli - času dne)
    posun_velikost = vzdalenost.copy()  
    posun_velikost[vzdalenost>0.1] = 0.1
    if (cas_pres_den >= 6*60) & (cas_pres_den <= 14*60):
        posun_velikost[vzdalenost<0.2] = 0
    else:
        posun_velikost[vzdalenost<0.05] = 0     
    # z jednotkového vektoru směru k cíli a velikosti posunu spočtěte celkový posun    
    posun[:,0] += posun_velikost*smer[:,0]
    posun[:,1] += posun_velikost*smer[:,1]
    return posun

Funkce **update** bude pak vypadat následovně:

In [ ]:
def update(cas_simulace_minuty):
    # volání nové funkce a posun pozic
    posun = do_prace_a_zpet(cas_simulace_minuty, pozice, domovy_lidi, prace_lidi)
    pozice[:,:] += posun
    
    # zbytek stejně jako v předchozích funkcích
    pozice[:, :] += (np.random.rand(pocet_lidi, 2) - 0.5) / 100   # hrubá simulace náhodného pohybu
    nakazili_se, nakazili_ostatni = nove_nakazeni(pozice, stav, vzdalenost_nakazy, pravdepodobnost_nakazy)
    nakazil_lidi[:] += nakazili_ostatni # přičtení kolik kdo nakazil nových lidí v tomto časovém kroce
    stav[nakazili_se] = 'I' # změna stavu nově nakažených
    
    doba_od_nakazeni[stav == 'I'] +=15 # inkrementujte dobu_od_nakazeni pro lidi se stavem 'I'
    po_nemoci = (doba_od_nakazeni >= delka_nakazy) & (stav == 'I') # nemocní a překonají delka_nakazy
    stav[po_nemoci] = 'R' # změní svůj stav na R
    
    #----------------------------------------------------------------------------------------
    vyber_poctu_nakazenych = nakazil_lidi[po_nemoci]
    if vyber_poctu_nakazenych.size == 0:
        R0 = -1
    else:
        R0 = vyber_poctu_nakazenych.mean()
    #----------------------------------------------------------------------------------------
    
    vykresleni(cas_simulace_minuty, pozice, stav, komponenty_grafu, R0) # R0 přidáme jako další parametr  
    return

In [ ]:
update(0)

Vyzkoušejte, jak vám to funguje:

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi, domovy_lidi, prace_lidi = reset_simulace(pocet_lidi, domovy, prace)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1, repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()

### Karanténa

Jak vidíte, tak model, ve kterém se lidi častěji dostávají do kontaktu, způsobí extrémní rychlost šíření nákazy. Jedním ze způsobů, jak s tím můžeme bojovat, je určení nakažených jedinců (např. testem) a jejich uzavření do karantény.

Proto zavedeme novou proměnnou našeho modelu **pravdepodobnost_zjisteni_a_karanteny**, která bude určovat pravděpodobnost, že po nakažení se daná tečka dostane do karantény, nemůže nikoho dalšího nakazit.

In [ ]:
pravdepodobnost_zjisteni_a_karanteny = 0.75

<hr style="border:1px solid black"> </hr>

## Úkol 8: 
Upravte model tak, aby osoby ve stavu **I** přešly se zadanou pravděpodobností **pravdepodobnost\_zjisteni\_a\_karanteny** do nově zavedeného stavu **Q** (osoba je v karanténě). Osoba ve stavu **Q** je izolována a nemůže nikoho nakazit. Počítejte však také s délkou nemoci člověka v karanténě a po nemoci ho přesuňte do stavu **R** stejně jako by byl nemocný.

<hr style="border:1px solid black"> </hr>

In [ ]:
def update(cas_simulace_minuty):
    # stejně jako v předchozích funkcích
    posun = do_prace_a_zpet(cas_simulace_minuty, pozice, domovy_lidi, prace_lidi)
    pozice[:,:] += posun
    pozice[:, :] += (np.random.rand(pocet_lidi, 2) - 0.5) / 100   # hrubá simulace náhodného pohybu
    nakazili_se, nakazili_ostatni = nove_nakazeni(pozice, stav, vzdalenost_nakazy, pravdepodobnost_nakazy)
    nakazil_lidi[:] += nakazili_ostatni # přičtení kolik kdo nakazil nových lidí v tomto časovém kroce
    stav[nakazili_se] = 'I' # změna stavu nově nakažených
    
    #----------------------------------------------------------------------------------------
    # zde přidejte změnu stavu na 'Q' pro pravdepodobnost_zjisteni_a_karanteny % nove nakazenych
    stav[nakazili_se & (np.random.rand(pocet_lidi) < pravdepodobnost_zjisteni_a_karanteny)] = 'Q'
    # pro tecky ve stavu 'Q' taky pocitejte dobu od nakazeni at mohou poté prejit do stavu 'R' 
    doba_od_nakazeni[stav == 'Q'] +=15
    #----------------------------------------------------------------------------------------
    
    # stejně jako v předchozích (až na drobnou úpravu po_nemoci, kde zohledňujeme také lidi v karanténě)   
    doba_od_nakazeni[stav == 'I'] +=15 # inkrementujte dobu_od_nakazeni pro lidi se stavem 'I'
    # nemocní nebo v karenténě a překonají delka_nakazy
    po_nemoci = (doba_od_nakazeni >= delka_nakazy) & ((stav == 'I') | (stav == 'Q')) 
    stav[po_nemoci] = 'R' # změní svůj stav na R
    
    vyber_poctu_nakazenych = nakazil_lidi[po_nemoci]
    if vyber_poctu_nakazenych.size == 0:
        R0 = -1
    else:
        R0 = vyber_poctu_nakazenych.mean()

    vykresleni(cas_simulace_minuty, pozice, stav, komponenty_grafu, R0) # R0 přidáme jako další parametr  
    return

In [ ]:
update(0)

Vyzkoušíme, co to dělá:

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi, domovy_lidi, prace_lidi = reset_simulace(pocet_lidi, domovy, prace)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0,delka_simulace,15), interval=1, repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()

### Lockdown / práce z domu

Poslední úpravou naší simulace bude vynucení některých teček, aby zůstaly doma. Budeme uvažovat novou proměnnou, která určí kolik % lidí přestane chodit do práce a zůstane doma.

In [ ]:
procento_homeoffice = 0.5

<hr style="border:1px solid black"> </hr>

## Úkol 9: 
Upravte funkci **reset_simulace** tak, aby **procento_homeoffice** % lidí měli jako své pracoviště nastaveno svůj domov.

<hr style="border:1px solid black"> </hr>

In [ ]:
def reset_simulace(pocet_lidi, domovy, prace, procento_homeoffice):
    # původní kód
    pozice = np.random.rand(pocet_lidi,2) # nové náhodné pozice
    doba_od_nakazeni = np.zeros((pocet_lidi)) # pole kde budeme udržovat dobu od prvotního nakažení
    nakazil_lidi = np.zeros((pocet_lidi)) # pole, kde budeme napočítávat kolik lidí kdo nakazil
    stav = np.full(pocet_lidi,'S') # inicializace stavu
    stav[0:4] = 'I'
    
    # np.random.randint má tři parametry - nejnižší číslo, nejvyšší číslo, počet hodnot
    domovy_lidi = domovy[np.random.randint(0, domovy.shape[0], pocet_lidi),:]
    prace_lidi = prace[np.random.randint(0, prace.shape[0], pocet_lidi), :]
    
    #----------------------------------------------------------------------------------------
    # zde doplňte, nejprve vygenerujte vektor identifikátorů True/False, jestli se to lidí týká
    kdo_pracuje_z_domu = np.random.rand(pocet_lidi) < procento_homeoffice
    # pak těm kterých ano (True) nastave práci jako jejich domov
    prace_lidi[kdo_pracuje_z_domu,:] = domovy_lidi[kdo_pracuje_z_domu,:]
    #----------------------------------------------------------------------------------------
    return pozice, stav, doba_od_nakazeni, nakazil_lidi, domovy_lidi, prace_lidi

Vyzkoušíme, jak simulace vypadá nyní:

In [ ]:
pozice, stav, doba_od_nakazeni, nakazil_lidi, domovy_lidi, prace_lidi = reset_simulace(pocet_lidi, domovy, prace, 
                                                                                       procento_homeoffice)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0, delka_simulace, 15), interval=1, repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()

### Konec

Tímto jsme dokončili naši zjednodušenou simulci šíření nákazy. To však neznamená, že si s ní nemůžete dále 'hrát'. Zkuste měnit jednotlivé parametry. Co podle vás nejvíce ovlivní rychlost šíření? 

In [ ]:
pocet_lidi=1000 #nezadavejte hodnoty vetsi nez 5000, simulace pak bude velmi pomala
delka_simulace=30*24*60 #30 dni * 24 hodin * 60 minut
vzdalenost_nakazy = 0.02 #vzdalenost, na kterou se mužeme nakazit
pravdepodobnost_nakazy = 0.01 #pravděpodobnost, že se nakazíme
delka_nakazy = 7*24*15 #délka nákazy
procento_lidi_pracujicich_z_domu = 0.3
pravdepodobnost_zjisteni_a_karanteny = 0.8

pozice, stav, doba_od_nakazeni, nakazil_lidi, domovy_lidi, prace_lidi = reset_simulace(pocet_lidi, domovy, prace, 
                                                                                       procento_homeoffice)
figure_animace, komponenty_grafu = nova_animace(pozice, stav, pocet_lidi, delka_simulace)
ani = FuncAnimation(figure_animace, update, frames=range(0, delka_simulace, 15), interval=1, repeat=False)
plt.show()

In [ ]:
ani.event_source.stop()